---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [1]:
import pandas as pd
import numpy as np
import nltk
spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [2]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [3]:
def answer_one():    
    return (len(spam_data[spam_data['target']==1])/len(spam_data)*100)#Your answer here

In [4]:
answer_one()

13.406317300789663

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [51]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
    vect = CountVectorizer().fit(X_train)
    return max(vect.get_feature_names(), key=len)#Your answer here

In [6]:
answer_two()

'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [32]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    
    vect = CountVectorizer().fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    model = MultinomialNB(alpha=0.1).fit(X_train_vectorized,y_train)
    predictions = model.predict(vect.transform(X_test))
    
    return roc_auc_score(y_test, predictions)#Your answer here

In [8]:
answer_three()

0.97208121827411165

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [178]:
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_four():
    
    vect = TfidfVectorizer().fit(X_train)
    feature_names = np.array(vect.get_feature_names())
    X_train_vectorized = vect.transform(X_train)
    sorted_tfidf_index = X_train_vectorized.max(0).toarray()[0].argsort()
    tfidf = X_train_vectorized.max(0).toarray()[0]
    
    large_df = pd.DataFrame(tfidf[sorted_tfidf_index[:-21:-1]],columns=['tfidf'])
    large_df['names'] = feature_names[sorted_tfidf_index[:-21:-1]]
    large_df = large_df.sort_values(['tfidf','names'])
    
    small_df = pd.DataFrame(tfidf[sorted_tfidf_index[:21]],columns=['tfidf'])
    small_df['names'] = feature_names[sorted_tfidf_index[:21]]
    small_df = small_df.sort_values(['tfidf','names'])
    small_tfidfs = pd.Series(small_df['tfidf'])
    small_tfidfs = small_tfidfs.reset_index().set_index(small_df['names']).drop(['index'],axis=1)
    large_tfidfs = pd.Series(large_df['tfidf'])
    large_tfidfs = large_tfidfs.reset_index().set_index(large_df['names']).drop(['index'],axis=1)
    
    return (small_df,pd.Series(large_tfidfs.tfidf))#Your answer here


In [179]:
answer_four()

(       tfidf         names
 2   0.074475        aaniye
 6   0.074475      athletic
 11  0.074475          chef
 4   0.074475     companion
 12  0.074475    courageous
 3   0.074475    dependable
 10  0.074475    determined
 7   0.074475  exterminator
 1   0.074475        healer
 5   0.074475      listener
 15  0.074475     organizer
 9   0.074475          pest
 8   0.074475  psychiatrist
 14  0.074475  psychologist
 16  0.074475       pudunga
 13  0.074475       stylist
 0   0.074475   sympathetic
 17  0.074475        venaam
 20  0.091250     childrens
 18  0.091250        diwali
 19  0.091250      mornings, names
 blank        0.932702
 tick         0.980166
 146tf150p    1.000000
 645          1.000000
 anything     1.000000
 anytime      1.000000
 beerage      1.000000
 done         1.000000
 er           1.000000
 havent       1.000000
 home         1.000000
 lei          1.000000
 nite         1.000000
 ok           1.000000
 okie         1.000000
 thank        1.000000
 thanx   

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [10]:
def answer_five():
    vect = TfidfVectorizer(min_df=3).fit(X_train)
    feature_names = np.array(vect.get_feature_names())
    X_train_vectorized = vect.transform(X_train)
    model = MultinomialNB(alpha=0.1).fit(X_train_vectorized,y_train)
    predictions = model.predict(vect.transform(X_test))
    
    return roc_auc_score(y_test, predictions)#Your answer here

In [13]:
answer_five()

0.94162436548223349

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [76]:
def answer_six():
    return ((sum([len(x) for x in spam_data[spam_data['target']!=1]['text']])/len(spam_data[spam_data['target']!=1])),(sum([len(x) for x in spam_data[spam_data['target']==1]['text']])/len(spam_data[spam_data['target']==1]))) #Your answer here

In [77]:
answer_six()

(71.02362694300518, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [33]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [36]:
from sklearn.svm import SVC

def answer_seven():
    vect = TfidfVectorizer(min_df=5).fit(X_train)
    feature_names = np.array(vect.get_feature_names())
    X_train_vectorized = vect.transform(X_train)
    
    X_train_new = add_feature(X_train_vectorized, [len(x) for x in X_train])
    model = SVC(C=10000).fit(X_train_new,y_train)
    predictions = model.predict(add_feature(vect.transform(X_test),[len(x) for x in X_test]))
    
    
    return roc_auc_score(y_test, predictions)#Your answer here

In [93]:
answer_seven()

0.95813668234215565

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [29]:
def answer_eight():
    sum_ns = 0
    for x in spam_data[spam_data['target']!=1]['text']:
        sum_ns += len(list(filter(str.isdigit, x)))

    sum_s = 0
    for x in spam_data[spam_data['target']==1]['text']:
        sum_s += len(list(filter(str.isdigit, x)))

    
    return ((sum_ns/len(spam_data[spam_data['target']!=1])),(sum_s/len(spam_data[spam_data['target']==1])))#Your answer here

In [30]:
answer_eight()

(0.2992746113989637, 15.759036144578314)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [79]:
from sklearn.linear_model import LogisticRegression
import re
def answer_nine():
    spam_data['len_docs'] = spam_data['text'].str.len()
    spam_data['dig_perdoc'] = spam_data['text'].apply(lambda row:len(re.findall(r'(\d)',row)))
    X_train, X_test, y_train, y_test = train_test_split(spam_data[['text', 'target', 'len_docs', 'dig_perdoc']],                                            spam_data['target'], random_state=0)

    vectorizer = TfidfVectorizer(min_df=5, ngram_range=(1,3)).fit(X_train['text'])
    X_train_vectorized = vectorizer.transform(X_train['text'])
    X_train_vectorized = add_feature(X_train_vectorized, X_train['len_docs'])
    X_train_vectorized = add_feature(X_train_vectorized, X_train['dig_perdoc'])
    X_test_vectorized = vectorizer.transform(X_test['text'])
    X_test_vectorized = add_feature(X_test_vectorized, X_test['len_docs'])
    X_test_vectorized = add_feature(X_test_vectorized, X_test['dig_perdoc'])
    
    model = LogisticRegression(C=100).fit(X_train_vectorized,y_train)
    predictions = model.predict(X_test_vectorized)
    return roc_auc_score(y_test, predictions)#Your answer here

In [80]:
answer_nine()

0.96533283533945646

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [88]:
def answer_ten():
    
    
    return ((sum(spam_data[spam_data['target']!=1]['text'].apply(lambda row:len(re.findall(r'(\W)',row))).values)/len(spam_data[spam_data['target']!=1])),(sum(spam_data[spam_data['target']==1]['text'].apply(lambda row:len(re.findall(r'(\W)',row))).values)/len(spam_data[spam_data['target']==1])))#Your answer here

In [89]:
answer_ten()

(14.58839378238342, 25.318607764390897)

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [181]:
def answer_eleven():
    
    spam_data['non_word_char_count'] = spam_data['text'].apply(lambda row:len(re.findall(r'(\W)',row)))
    spam_data['length_of_doc'] = spam_data['text'].str.len()
    spam_data['digit_count'] = spam_data['text'].apply(lambda row:len(re.findall(r'(\d)',row)))
    X_train, X_test, y_train, y_test = train_test_split(spam_data[['text', 'target', 'length_of_doc', 'digit_count', 'non_word_char_count']],
                                                        spam_data['target'], random_state=0)

    vectorizer = CountVectorizer(min_df=5, ngram_range=(2,5),analyzer='char_wb').fit(X_train['text'])
    X_train_vectorized = vectorizer.transform(X_train['text'])
    X_train_vectorized = add_feature(X_train_vectorized, X_train['length_of_doc'])
    X_train_vectorized = add_feature(X_train_vectorized, X_train['digit_count'])
    X_train_vectorized = add_feature(X_train_vectorized, X_train['non_word_char_count'])
    
    X_test_vectorized = vectorizer.transform(X_test['text'])
    X_test_vectorized = add_feature(X_test_vectorized, X_test['length_of_doc'])
    X_test_vectorized = add_feature(X_test_vectorized, X_test['digit_count'])
    X_test_vectorized = add_feature(X_test_vectorized, X_test['non_word_char_count'])
    
    model = LogisticRegression(C=100).fit(X_train_vectorized,y_train)
    predictions = model.predict(X_test_vectorized)
    feature_names = list(vectorizer.get_feature_names()+['length_of_doc', 'digit_count', 'non_word_char_count'])
    feature_names = pd.Series(feature_names)
    update_coefs = (model.coef_[0]).argsort()
    feature_names = feature_names[update_coefs]
    return (roc_auc_score(y_test, predictions),pd.Series(model.coef_[0][model.coef_[0].argsort()][:10], index=feature_names[:10]).sort_values(),pd.Series(model.coef_[0][model.coef_[0].argsort()][:-11:-1],index=feature_names[:-11:-1]))#Your answer here

In [182]:
answer_eleven()

(0.97885931107074342, .     -0.869745
 ..    -0.860867
 ?     -0.676970
  i    -0.667010
  y    -0.614893
  go   -0.579597
 :)    -0.535072
  h    -0.505765
 go    -0.498514
  m    -0.490946
 dtype: float64, digit_count    1.212221
 ne             0.597776
 ia             0.541486
 co             0.538758
 xt             0.521478
  ch            0.520357
 mob            0.517867
  x             0.516092
 ww             0.508665
 ar             0.502649
 dtype: float64)